In [280]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../dataset/meas_asymm_nomove_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 10  # Try values like 20, 50, or 100
n_nonzero_coefs = 5 # Sparsity level

In [281]:
def create_sparse_representation(data, n_components, n_nonzero_coefs):
    """
    Create a sparse representation with reduced dimensions.
    
    Parameters:
    - data: Input data with shape (n_samples, n_data_points, n_features)
    - n_components: Reduced dimensionality
    - n_nonzero_coefs: Sparsity level
    
    Returns:
    - Sparse representation with reduced dimensions.
    """
    n_samples, n_data_points, n_features = data.shape

    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_features))

    # Process each feature separately
    for feature in range(n_features):
        # Extract data for current feature
        feature_data = data[:, :, feature]  # Shape: (n_samples, n_data_points)

        # Learn a dictionary (or use a pre-defined one)
        dict_learner = DictionaryLearning(
            n_components=n_components, 
            transform_algorithm='lasso_lars', 
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        dict_learner.fit(feature_data)
        sparse_codes = dict_learner.transform(feature_data)

        # Store the sparse codes for this feature
        sparse_representation[:, :, feature] = sparse_codes

    return sparse_representation

In [282]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# Learn the dictionary
dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', transform_n_nonzero_coefs=n_nonzero_coefs, n_jobs= -1)
dict_learner.fit(train_data_combined)
# D = dict_learner.components_  # Shape: (n_components, feature_dim)


train_alice_sparse_codes = dict_learner.transform(train_alice_magnitude)
train_eve_sparse_codes = dict_learner.transform(train_eve_magnitude)
train_sparse_codes = np.vstack((train_alice_sparse_codes, train_eve_sparse_codes))
print(train_alice_sparse_codes.shape)
print(train_alice_sparse_codes[0])

(800, 10)
[49128.55566909 -3821.45888911  6381.15835743 -9042.81129299
 10380.35838822  4963.07154832   150.51064708  9267.77127048
  1753.77742528  -225.00391632]


In [283]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]

# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))


# Transform test data into sparse codes
test_alice_sparse_codes = dict_learner.transform(test_alice_magnitude)
test_eve_sparse_codes = dict_learner.transform(test_eve_magnitude)

# Combine the sparse codes
test_sparse_codes = np.vstack((test_alice_sparse_codes, test_eve_sparse_codes))
print('Test data shape:', test_sparse_codes.shape)

Test data shape: (400, 10)


In [284]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_sparse_codes.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_sparse_codes.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_sparse_codes.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_sparse_codes.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))


In [285]:
# # Dictionary
# D = train_sparse_codes.T
# print('Dictionary shape:', D.shape)

In [286]:
scaler = StandardScaler()
train_sparse_codes = scaler.fit_transform(train_sparse_codes)
# Apply the same scaling to test data
test_sparse_codes = scaler.transform(test_sparse_codes)
print('Train data shape:', train_sparse_codes.shape)

Train data shape: (1600, 10)


In [287]:
# Initialize and train the SVM classifier
classifier = SVC(kernel='rbf', random_state=42)
classifier.fit(train_sparse_codes, train_labels)

# Predict on test data
predictions = classifier.predict(test_sparse_codes)

In [288]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 100.00%
tp: 200
tn: 200
fp: 0
fn: 0
MDR: 0.0
FAR: 0.0
AR: 1.0
